# Implementation: Bottleneck Adapter

**Goal**: Inject a tiny trainable layer into a frozen block.

In [ ]:
import torch
import torch.nn as nn

class FrozenLayer(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(100, 100)
        # Freeze immediately
        for p in self.parameters():
            p.requires_grad = False
            
    def forward(self, x):
        return self.linear(x)

class Adapter(nn.Module):
    def __init__(self, input_dim, bottleneck_dim):
        super().__init__()
        self.down = nn.Linear(input_dim, bottleneck_dim)
        self.act = nn.ReLU()
        self.up = nn.Linear(bottleneck_dim, input_dim)
        
    def forward(self, x):
        return self.up(self.act(self.down(x)))

# 1. Compose
frozen_block = FrozenLayer()
adapter = Adapter(input_dim=100, bottleneck_dim=4)

# 2. Forward with Skip Connection
x = torch.randn(1, 100)

# Output = Freezed(x) + Adapter(x)
out = frozen_block(x) + adapter(x)

print(f"Output Shape: {out.shape}")

trainable = sum(p.numel() for p in adapter.parameters())
frozen = sum(p.numel() for p in frozen_block.parameters())

print(f"Frozen Params: {frozen}")
print(f"Trainable Params: {trainable} (Tiny!)")

## Conclusion
We added flexibility to the model using only ~800 parameters, while the main layer has 10,000.